# Агент с текстовым поиском на основе Yandex Search API

В данном примере рассмотрим возможность построения агента с использованием текстового поиска на основе [Yandex Search API](https://yandex.cloud/ru/services/search-api)  

Для данного примера необходимо:
- **GigaChat API**: получить [авторизационные данные](https://developers.sber.ru/docs/ru/gigachain/get-started/quickstart#nastroyka-avtorizatsii-zaprosov-k-giga-chat-api)
- **Yandex Search API**: получить идентификатор каталога (folderid) и API-ключ (apikey) для [аутентификации в сервисе](https://yandex.cloud/ru/docs/search-api/operations/auth)

In [1]:
from getpass import getpass
import os
from uuid import uuid4

from dotenv import find_dotenv, load_dotenv

In [2]:
def _set_env(key: str):
    if key not in os.environ:
        os.environ[key] = getpass(f"{key}:")

In [3]:
load_dotenv(find_dotenv())

True

In [4]:
_set_env("GIGACHAT_CREDENTIALS")
_set_env("YANDEX_API_KEY")
_set_env("YANDEX_FOLDER_ID")

## Конфигурация

In [5]:
from operator import itemgetter
from textwrap import dedent

from IPython.display import Markdown
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_gigachat.chat_models import GigaChat
from langchain_community.tools.yandex_search import YandexSearchResults
from langchain_community.utilities.yandex_search import YandexSearchAPIWrapper
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables.history import RunnableWithMessageHistory

Используем модель `GigaChat-Pro` для данной задачи.

In [6]:
model = GigaChat(
    model="GigaChat-Pro",
    verify_ssl_certs=False,
)

Создадим клиента для Yandex Search API с параметрами по умолчанию.  
Детальную информацию по дополнительным параметрам можно уточнить в [документации](https://yandex.cloud/ru/docs/search-api/concepts/get-request).

In [8]:
api_wrapper = YandexSearchAPIWrapper()

Определим tool для агента для возможности поиска в интернете.  
В случае, если клиент для Yandex Search API используется с параметрами по умолчанию, то можно не передавать аргумент api_wrapper для tool.

In [9]:
search_tool = YandexSearchResults(max_results=10)

Создадим агента, предоставив ему tool для возможности поиска в интернете в случае необходимости.

In [10]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

tools = [search_tool]

agent = create_tool_calling_agent(model, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

session_id = str(uuid4())
memory = InMemoryChatMessageHistory(session_id=session_id)
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    lambda session_id: memory,
    input_messages_key="input",
    history_messages_key="chat_history",
)
config = {"configurable": {"session_id": session_id}}

## Примеры использования

In [11]:
while True:
    user_input = input("User:")
    print(f"User: {user_input}")
    
    if user_input.lower() in ["quit", "exit", "q"]:
        print("До свидания!")
        break
        
    response = agent_with_chat_history.invoke({"input": user_input}, config)['output']
    print(f"Agent: {response}")

User: Привет! Как дела?


User: Привет! Как дела?


> Entering new AgentExecutor chain...
Здравствуйте! Я генеративная языковая модель, поэтому у меня не может быть собственных дел. Но я всегда готова ответить на ваши вопросы или поддержать беседу.

> Finished chain.
Agent: Здравствуйте! Я генеративная языковая модель, поэтому у меня не может быть собственных дел. Но я всегда готова ответить на ваши вопросы или поддержать беседу.


User: Почему небо голубое?


User: Почему небо голубое?


> Entering new AgentExecutor chain...
На самом деле цвет неба определяется рассеянием света в атмосфере. Солнечный свет состоит из волн разной длины, которые воспринимаются нашим глазом как разные цвета (красный, оранжевый, желтый, зеленый, голубой, синий, фиолетовый). Короткие волны, а именно голубые и синие, рассеиваются сильнее всего. Поэтому мы видим небо голубым.

> Finished chain.
Agent: На самом деле цвет неба определяется рассеянием света в атмосфере. Солнечный свет состоит из волн разной длины, которые воспринимаются нашим глазом как разные цвета (красный, оранжевый, желтый, зеленый, голубой, синий, фиолетовый). Короткие волны, а именно голубые и синие, рассеиваются сильнее всего. Поэтому мы видим небо голубым.


User: Кто выиграл ЕВРО-2024?


User: Кто выиграл ЕВРО-2024?


> Entering new AgentExecutor chain...

Invoking: `yandex_search_results_json` with `{'query': 'кто выиграл евро-2024'}`


[{'url': 'https://meduza.io/feature/2024/07/15/evro-2024-zakonchilsya-ispaniya-chempion', 'content': '14 июля сборная Испании в финале Евро-2024 обыграла Англию со счетом 2:1 и в четвертый раз стала чемпионом Европы по футболу (это рекорд!) Финальный матч проходил на Олимпийском стадионе в Берлине. Болельщики обеих сборных следили за игрой в самых разных местах — в пабах, кафе, фан-зонах, на улицах и стадионах.'}, {'url': 'https://ru.wikipedia.org/wiki/%D0%A7%D0%B5%D0%BC%D0%BF%D0%B8%D0%BE%D0%BD%D0%B0%D1%82_%D0%95%D0%B2%D1%80%D0%BE%D0%BF%D1%8B_%D0%BF%D0%BE_%D1%84%D1%83%D1%82%D0%B1%D0%BE%D0%BB%D1%83_2024', 'content': 'Чемпионат Европы по футболу 2024 года (англ. 2024 UEFA European Football Championship или Евро-2024, UEFA Euro 2024 нем.'}, {'url': 'https://www.nur.kz/sport/football/2133837-sbornaya-ispanii-obygrala-angliyu-i-stala-chempi

User: Спасибо!


User: Спасибо!


> Entering new AgentExecutor chain...
Пожалуйста! Если у Вас есть ещё вопросы, задавайте.

> Finished chain.
Agent: Пожалуйста! Если у Вас есть ещё вопросы, задавайте.


User: q


User: q
До свидания!
